In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
from sklearn.model_selection import train_test_split

from model import build_model
from load_data import get_patches_path, data_generate
from loss import bag_loss, bag_accuracy

print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))

INPUT_PATCH_FOLDER = "data/train_imgs_patch"
PATCH_WIDTH = 128
PATCH_SHAPE = (PATCH_WIDTH, PATCH_WIDTH, 3)
BATCH_SIZE = 1

In [ ]:
clinical_data = pd.read_csv("data/train.csv")
bag_names = list(clinical_data["ID"])
labels = list(clinical_data["N_category"])
patch_bags = get_patches_path(INPUT_PATCH_FOLDER)

In [ ]:
(
    train_bag_names,
    val_bag_names,
    train_y,
    val_y,
    train_bags,
    val_bags,
) = train_test_split(
    bag_names[: len(patch_bags)], labels[: len(patch_bags)], patch_bags
)

In [ ]:
train_dataset = tf.data.Dataset.from_generator(
    generator=data_generate,
    output_types=(tf.float32, tf.float32),
    output_shapes=(
        tf.TensorShape([None, PATCH_WIDTH, PATCH_WIDTH, 3]),
        tf.TensorShape(
            [
                1, 1
            ]
        ),
    ),
    args=(train_bag_names, train_y, train_bags),
)

val_dataset = tf.data.Dataset.from_generator(
    generator=data_generate,
    output_types=(tf.float32, tf.float32),
    output_shapes=(
        tf.TensorShape([None, PATCH_WIDTH, PATCH_WIDTH, 3]),
        tf.TensorShape(
            [
                1, 1
            ]
        ),
    ),
    args=(val_bag_names, val_y, val_bags),
)

In [ ]:
model = build_model(PATCH_SHAPE)
model.summary()

In [ ]:
os.makedirs("check_points", exist_ok=True)
model_name = "check_points/" + "acc({accuracy:.4f})" + "epoch({epoch})" + "val_loss({val_loss:.4f}).hd5"
check_point = tf.keras.callbacks.ModelCheckpoint(
    model_name,
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    mode="auto",
)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)
callbacks = [check_point, early_stopping]

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=bag_loss,
    metrics=[bag_accuracy],
)

model.fit(
    train_dataset, 
    validation_data=val_dataset, 
    callbacks=callbacks, 
    epochs=10,
    steps_per_epoch=len(train_bags)//BATCH_SIZE,
    validation_steps=len(val_bags)//BATCH_SIZE
)
